In [1]:
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.jit as jit
import warnings
from collections import namedtuple
from typing import List, Tuple
from torch import Tensor
import numbers
import numpy as np

In [2]:
input_dim = 4
hidden_dim = 3

In [9]:
class LSTMCell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.n_gates = 6
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.weight_ih = Parameter(torch.randn(self.n_gates * hidden_size, input_size))
        self.weight_hh = Parameter(torch.randn(self.n_gates * hidden_size, hidden_size))
        self.bias_ih = Parameter(torch.randn(self.n_gates * hidden_size))
        self.bias_hh = Parameter(torch.randn(self.n_gates * hidden_size))

    def forward(self, input: Tensor, state: Tuple[Tensor, Tensor, Tensor]) -> Tuple[Tensor, Tuple[Tensor, Tensor]]:
        hx, cx1, cx2 = state
        
        gates = (torch.matmul(input, self.weight_ih.t()) + self.bias_ih +
                 torch.matmul(hx, self.weight_hh.t()) + self.bias_hh)
        
        ingate1, ingate2, forgetgate1, forgetgate2, cellgate, outgate = gates.chunk(self.n_gates)

        ingate1 = torch.sigmoid(ingate1)
        ingate2 = torch.sigmoid(ingate2)
        forgetgate1 = torch.sigmoid(forgetgate1)
        forgetgate2 = torch.sigmoid(forgetgate2)
        cellgate = torch.tanh(cellgate)
        outgate = torch.sigmoid(outgate)
        

        cy1 = (forgetgate1 * cx1) + (ingate1 * cellgate)
        cy2 = (forgetgate2 * cx2) + (ingate2 * cellgate)
        hy = outgate * torch.tanh(cy)

        return hy, (hy, cy1, cy2)

# train

In [14]:
model = combined(input_dim, hidden_dim)

for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        

loss_function = nn.CrossEntropyLoss()   
optimizer = torch.optim.Adam(model.parameters(),lr=0.01)  
epochs=500
final_losses=[]

for i in range(epochs):
    i= i+1

    x = model.forward(input1, (a, b))

    loss=loss_function(x , y_test)

    final_losses.append(loss)

    if i % 100 == 1:

        print("Epoch number: {} and the loss : {}".format(i,loss.item()))

    optimizer.zero_grad()

    loss.backward()
    
    optimizer.step()
    
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        

cell1.weight_ih tensor([[ 1.0475, -1.2576,  1.4938, -0.1403],
        [-0.1761,  0.2446, -0.0066,  0.0169],
        [ 0.6526, -0.3397, -0.0401,  0.2991],
        [-0.8911, -0.1530,  0.4985,  0.1815],
        [ 0.8046, -0.3014, -0.1771,  0.5974],
        [ 1.9203, -2.3702,  1.0451, -1.3581],
        [-0.5414,  0.9392,  0.4592,  0.1241],
        [ 0.5214, -0.8826,  2.0588, -0.1296],
        [ 0.9739,  0.3298, -1.9964,  1.2978],
        [ 0.6754, -0.5145, -1.5474, -0.3484],
        [ 0.2155, -0.7136, -0.7659,  0.3172],
        [ 0.9694,  1.2144,  1.5976,  0.9273],
        [-0.0898,  0.1910,  2.2066,  1.9341],
        [ 2.2889, -1.9971,  1.0205,  1.2434],
        [ 0.0137,  0.4146, -0.0572, -1.7984],
        [ 0.5057, -1.2772, -1.9820,  0.5453],
        [-0.4307, -1.6967, -0.6890,  0.2548],
        [ 0.6678,  0.2351, -0.7527,  0.1685]])
cell1.weight_hh tensor([[ 1.2539, -0.3586, -1.7616],
        [-0.9085,  0.3503,  0.1402],
        [ 0.7802,  0.1413,  0.2270],
        [ 1.4114, -0.4574,  

ValueError: not enough values to unpack (expected 3, got 2)

In [97]:
model = LSTMCell(1, 16)
test_state = (torch.randn(16), torch.randn(16), torch.randn(16))
a = (torch.randn(16), torch.randn(16), torch.randn(16))

In [98]:
model.forward(a)

NameError: name 'hy' is not defined

In [99]:
class LSTMLayer(nn.Module):
    def __init__(self, cell, *cell_args)

1


In [120]:
class LSTMCellUnknownData(nn.Module):
    def __init__(self, input_size, hidden_size = 16):
        super().__init__()
        self.Tanh = nn.Tanh()
        self.Softplus = nn.Softplus()
        
        self.LC_i1 = Gate_network(input_size, hidden_size)
        self.LC_i2 = Gate_network(input_size, hidden_size)
        self.LC_f1 = Gate_network(input_size, hidden_size)
        self.LC_f2 = Gate_network(input_size, hidden_size)
        self.LC_o = Gate_network(input_size, hidden_size)
        self.LC_z = Gate_network(input_size, hidden_size)
        self.LC_d = Decay_network(input_size, hidden_size)
        self.LC_lambda = Decode_layer(input_size, hidden_size)
        
        self.tj = None
        self.cy1 = None
        self.cy2 = None
        self.decay_coef = None
        self.o = None
        
    def get_intensity(self, t):
        h_t = self.get_h(t)
        intensity = self.LC_lambda(h_t)
        
    def get_h(self, t):
        c_t = self.calc_c(t)
        h_t = self.o * torch.tanh(c_t)
        return h_t

    def get_c(self, t):
        c_t = self.cy2 + (self.cy1 - self.cy2)*torch.exp(-self.decay_coef*(t - tj))
        return c_t
        
    def forward(self, state):
        self.check_if_tj_is_set()
        
        hx, cx1, cx2 = state
        
        i1 = self.LC_i1(hx)
        i2 = self.LC_i2(hx)
        f1 = self.LC_f1(hx)
        f2 = self.LC_f2(hx)
        o = self.LC_o(hx)
        z = self.Tanh(self.LC_z(hx))

        self.o = o
        self.cy1 = (f1 * cx1) + (i1 * z)
        self.cy2 = (f2 * cx2) + (i2 * z)
        self.decay_coef = self.LC_d(hx)
        
        return self.cy2
    
    def set_tj(self, tj):
        self.tj = tj
        
    def check_if_tj_is_set(self):
        if self.tj == None:
            raise Exception('Cannot run forward of LSTM cell before its starting time is set. Use set_tj to set tj.')
            
class LSTMCellKnownData(nn.Module):
    def __init__(self, time_interval, input_size, hidden_size = 16):
        super().__init__()
        self.Tanh = nn.Tanh()
        self.Softplus = nn.Softplus()
        
        self.LC_i1 = Gate_network(input_size, hidden_size)
        self.LC_i2 = Gate_network(input_size, hidden_size)
        self.LC_f1 = Gate_network(input_size, hidden_size)
        self.LC_f2 = Gate_network(input_size, hidden_size)
        self.LC_o = Gate_network(input_size, hidden_size)
        self.LC_z = Gate_network(input_size, hidden_size)
        self.LC_d = Decay_network(input_size, hidden_size)
        self.LC_lambda = Decode_layer(input_size, hidden_size)
        
        self.t_start, self.t_end = time(interval)
        self.cy1 = None
        self.cy2 = None
        self.decay_coef = None
        self.o = None
        
    def get_intensity(self, h_t):
        intensity = self.LC_lambda(h_t)
        return intensity
        
    def get_h(self, c_t):
        h_t = self.o * torch.tanh(c_t)
        return h_t

    def get_c(self):
        c_t = self.cy2 + (self.cy1 - self.cy2)*torch.exp(-self.decay_coef*(self.t_end - self.t_start))
        return c_t
        
    def forward(self, state):
        self.check_if_tj_is_set()
        
        hx, cx1, cx2 = state
        
        i1 = self.LC_i1(hx)
        i2 = self.LC_i2(hx)
        f1 = self.LC_f1(hx)
        f2 = self.LC_f2(hx)
        o = self.LC_o(hx)
        z = self.Tanh(self.LC_z(hx))

        self.o = o
        self.cy1 = (f1 * cx1) + (i1 * z)
        self.cy2 = (f2 * cx2) + (i2 * z)
        self.decay_coef = self.LC_d(hx)
        
        c_output = self.get_c()
        h_output = self.get_h(c_output)
        intensity_output = self.get_intensity(h_output)
        
        state = (h_output, c_output, cy2)
        
        return intensity_output, state
    
    def set_tj(self, tj):
        self.tj = tj
        
    def check_if_tj_is_set(self):
        if self.tj == None:
            raise Exception('Cannot run forward of LSTM cell before its starting time is set. Use set_tj to set tj.')

In [83]:
class Gate_network(nn.Module):
    def __init__(self, input_size, hidden_size = 16):
        super().__init__()
        self.Tanh = nn.Tanh()
        self.Sigmoid = nn.Sigmoid()
        
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, input: Tensor):
        x = self.Tanh(self.layer1(input))
        x = self.Tanh(self.layer2(x))
        return self.Sigmoid(x)
    
class Decay_network(nn.Module):
    def __init__(self, input_size, hidden_size = 16):
        super().__init__()
        self.Softplus = nn.Softplus()
        
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, hidden_size)
        
    def forward(self, input: Tensor):
        x = self.Softplus(self.layer1(input))
        x = self.Softplus(self.layer2(x))
        return x
    
class Decode_layer(nn.Module):
    def __init__(self, input_size, hidden_size = 16):
        super().__init__()
        self.Softplus = nn.Softplus()
        
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, 1)
        
    def forward(self, input: Tensor):
        x = self.Softplus(self.layer1(input))
        x = self.Softplus(self.output_layer(x))
        return x

class Embed_layer(nn.Module):
    def __init__(self, input_size, hidden_size = 16):
        super().__init__()
        self.Tanh = nn.Tanh
        
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(input_size, hidden_size)
        
    def forward(self, input: Tensor):
        x = self.Tanh(self.layer1(input))
        x = self.Tanh(self.layer2(x))
        return x

In [111]:
class Test_layer(nn.Module):
    def __init__(self, input_size, hidden_size = 16):
        super().__init__()
        self.Softplus = nn.Softplus()
        
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.output_layer = nn.Linear(hidden_size, input_size)
        
        self.time_layer = nn.Linear(2, 2)
        
    def forward(self, input: Tensor, time):
        x = self.Softplus(self.layer1(input * time))
        x = self.Softplus(self.output_layer(x))
        return x
    
    def get_time(self, t):
        time = self.Softplus(self.time_layer(t))
        return time
        

In [112]:
class Test_framework(nn.Module):
    def __init__(self, input_size, hidden_size = 5):
        super().__init__()
        
        self.test1 = Test_layer(2, 3)
        self.test2 = Test_layer(2, 3)
        
    def forward(self, input, time):
        x = self.test1(input, time)
        time = self.test1.get_time(torch.randn(2))
        x = self.test2(x, time)
        return x

In [113]:
model = Test_framework(2)
model.forward(torch.randn(2), torch.randn(2))

tensor([0.9269, 0.6812], grad_fn=<SoftplusBackward0>)

In [114]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        

test1.layer1.weight tensor([[-0.3199,  0.6786],
        [ 0.1927,  0.3247],
        [ 0.5788, -0.6896]])
test1.layer1.bias tensor([ 0.1387, -0.2884, -0.3352])
test1.output_layer.weight tensor([[ 0.5773, -0.3392, -0.4058],
        [-0.5206, -0.1302, -0.3975]])
test1.output_layer.bias tensor([-0.0074,  0.0397])
test1.time_layer.weight tensor([[ 0.6922,  0.2373],
        [ 0.5635, -0.2160]])
test1.time_layer.bias tensor([-0.2433, -0.1554])
test2.layer1.weight tensor([[ 0.5514, -0.6544],
        [-0.3557,  0.0063],
        [ 0.5027, -0.0073]])
test2.layer1.bias tensor([-0.6543,  0.4370, -0.4045])
test2.output_layer.weight tensor([[-0.0213,  0.0353, -0.0183],
        [ 0.1831,  0.0750, -0.0770]])
test2.output_layer.bias tensor([ 0.4107, -0.1227])
test2.time_layer.weight tensor([[ 0.0657, -0.2485],
        [-0.0894,  0.5229]])
test2.time_layer.bias tensor([-0.2745, -0.3383])
